# SQL Task: Product Table Comparison

In [ ]:

import pandas as pd
import sqlite3

# Load CSVs
df1 = pd.read_csv('products_day1.csv')
df2 = pd.read_csv('products_day2.csv')

# Create SQLite in-memory database
conn = sqlite3.connect(':memory:')

df1.to_sql('products_day1', conn, index=False, if_exists='replace')
df2.to_sql('products_day2', conn, index=False, if_exists='replace')

# Find Added/Removed Rows
added_removed_query = '''
SELECT *,'REMOVED' as change_type FROM products_day1
WHERE product_id NOT IN (SELECT product_id FROM products_day2)
UNION ALL
SELECT *,'ADDED' as change_type FROM products_day2
WHERE product_id NOT IN (SELECT product_id FROM products_day1)
'''
added_removed_df = pd.read_sql_query(added_removed_query, conn)
added_removed_df.to_csv('added_removed_rows.csv', index=False)
added_removed_df


In [ ]:

# Find Column-Level Changes
compare_query = '''
SELECT d1.product_id, 
       'price' AS column_changed, 
       d1.price AS old_value, 
       d2.price AS new_value
FROM products_day1 d1
JOIN products_day2 d2 ON d1.product_id = d2.product_id
WHERE d1.price != d2.price

UNION ALL

SELECT d1.product_id, 
       'stock' AS column_changed, 
       d1.stock AS old_value, 
       d2.stock AS new_value
FROM products_day1 d1
JOIN products_day2 d2 ON d1.product_id = d2.product_id
WHERE d1.stock != d2.stock
'''
column_diff_df = pd.read_sql_query(compare_query, conn)
column_diff_df.to_csv('column_level_diff.csv', index=False)
column_diff_df
